In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D
from keras.optimizers import Adam

from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import load_img, img_to_array

In [10]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [7]:
labels_all=pd.read_csv("targetdir/labels.csv")
print(labels_all.shape)
labels_all.head()

(10222, 2)


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [8]:
breeds_all=labels_all["breed"]
breed_count=breeds_all.value_counts()
breed_count.head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

In [9]:
class_names=["scottish_deerhound","maltese_dog","afghan_hound"]
labels=labels_all[(labels_all['breed'].isin(class_names))]
labels=labels.reset_index()
labels.head()

,index,id,breed
0,9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
1,12,00693b8bc2470375cc744a6391d397ec,maltese_dog
2,79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
3,90,022b34fd8734b39995a9f38a4f3e7b6b,maltese_dog
4,146,0379145880ad3978f9b80f0dc2c03fba,afghan_hound


In [10]:
X_data=np.zeros((len(labels),224,224,3),dtype='float32')
Y_data=label_binarize(labels['breed'],classes=class_names)

for i in tqdm(range(len(labels))):
    img=load_img('targetdir/train/%s.jpg' %labels['id'][i],target_size=(224,224))
    img=img_to_array(img)
    x=np.expand_dims(img.copy(),axis=0)
    X_data[i]= x / 255
print('\nTRAIN IMAGES shape:',X_data.shape,' size:{:,}',format(X_data.size))
print('\none hot encoded output shape:',Y_data.shape,' size:{:,}',format(Y_data.size))

100%|██████████| 359/359 [00:05<00:00, 62.01it/s]


TRAIN IMAGES shape: (359, 224, 224, 3)  size:{:,} 54039552

one hot encoded output shape: (359, 3)  size:{:,} 1077


In [11]:
model=Sequential()

# model.add(Conv2D(filters=128,kernel_size=(5,5),activation='relu',input_shape=(224,224,3)))
# model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(5,5),activation='relu',input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(5,5),activation='relu',kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=16,kernel_size=(5,5),activation='relu',kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=8,kernel_size=(5,5),activation='relu',kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(128,activation="relu",kernel_regularizer='l2'))
model.add(Dense(64,activation="relu",kernel_regularizer='l2'))
model.add(Dense(len(class_names),activation="softmax"))

model.compile(loss= 'categorial_crossentropy',optimizer=Adam(0.0001), metrics=['accuracy'])
          
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 32)      51232     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 53, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 49, 49, 16)        12816     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 24, 16)       0

In [12]:
X_train_and_val,X_test,Y_train_and_val,Y_test=train_test_split(X_data,Y_data,test_size=0.1)
X_train,X_val,Y_train,Y_val=train_test_split(X_train_and_val,Y_train_and_val,test_size=0.2)

In [23]:
epochs=100
batch_size=128

h=model.fit(X_train,Y_train,epochs=100,batch_size=128,validation_data=(X_val,Y_val))

Epoch 1/100


ValueError: in user code:

    File "C:\Users\Lenovo\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Lenovo\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Lenovo\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Lenovo\anaconda3\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Lenovo\anaconda3\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Lenovo\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 184, in __call__
        self.build(y_pred)
    File "C:\Users\Lenovo\anaconda3\l